# Ticketmaster - Event Data Fetching

## **event end time/duration**

In [ ]:
from google.colab import userdata

import requests
import pandas as pd
import json
from datetime import datetime
from IPython.display import display, HTML
import os
print(f"Current working directory: {os.getcwd()}")
api_key = userdata.get('TICKETMASTER_API_KEY')
base_url = "https://app.ticketmaster.com/discovery/v2/events.json"

def fetch_events(
    # Basic parameters
    country_code=None,
    size=20,
    page=0,
    sort="date,asc",

    # Identification parameters
    id=None,
    keyword=None,
    attraction_id=None,
    venue_id=None,

    # Geographic parameters
    city=None,
    state_code=None,
    postal_code=None,
    latlong=None,
    geo_point=None,
    radius=None,
    unit="miles",
    dma_id=None,

    # Date and time parameters
    start_date_time=None,
    end_date_time=None,
    local_start_date_time=None,
    local_start_end_date_time=None,
    start_end_date_time=None,

    # Classification parameters
    classification_name=None,
    classification_id=None,
    segment_id=None,
    segment_name=None,
    genre_id=None,
    sub_genre_id=None,
    type_id=None,
    sub_type_id=None,

    # Sale parameters
    onsale_start_date_time=None,
    onsale_end_date_time=None,
    onsale_on_start_date=None,
    onsale_on_after_start_date=None,
    pre_sale_date_time=None,
    public_visibility_start_date_time=None,

    # Include/exclude parameters
    include_tba="yes",
    include_tbd="yes",
    include_test="no",
    include_family="yes",
    include_spellcheck="no",

    # Other filtering parameters
    market_id=None,
    promoter_id=None,
    collection_id=None,
    source=None,
    locale="en",
    domain=None,
    preferred_country="us"
):
    # Define parameters dict with base parameters
    params = {
        'apikey': api_key,
        'size': size,
        'page': page,
        'sort': sort,
        'includeTBA': include_tba,
        'includeTBD': include_tbd,
        'includeTest': include_test,
        'includeFamily': include_family,
        'includeSpellcheck': include_spellcheck,
        'locale': locale,
        'preferredCountry': preferred_country
    }

    # Helper function to add parameters if they're provided
    def add_param(param_name, value, api_param_name=None):
        if value is not None:
            params[api_param_name or param_name] = value

    # Add identification parameters
    add_param('id', id)
    add_param('keyword', keyword)
    add_param('attractionId', attraction_id)
    add_param('venueId', venue_id)

    # Add geographic parameters
    add_param('countryCode', country_code)
    add_param('city', city)
    add_param('stateCode', state_code)
    add_param('postalCode', postal_code)
    add_param('latlong', latlong)
    add_param('geoPoint', geo_point)
    add_param('radius', radius)
    add_param('unit', unit)
    add_param('dmaId', dma_id)

    # Add date/time parameters
    add_param('startDateTime', start_date_time)
    add_param('endDateTime', end_date_time)
    add_param('localStartDateTime', local_start_date_time)
    add_param('localStartEndDateTime', local_start_end_date_time)
    add_param('startEndDateTime', start_end_date_time)

    # Add classification parameters
    add_param('classificationName', classification_name)
    add_param('classificationId', classification_id)
    add_param('segmentId', segment_id)
    add_param('segmentName', segment_name)
    add_param('genreId', genre_id)
    add_param('subGenreId', sub_genre_id)
    add_param('typeId', type_id)
    add_param('subTypeId', sub_type_id)

    # Add sale parameters
    add_param('onsaleStartDateTime', onsale_start_date_time)
    add_param('onsaleEndDateTime', onsale_end_date_time)
    add_param('onsaleOnStartDate', onsale_on_start_date)
    add_param('onsaleOnAfterStartDate', onsale_on_after_start_date)
    add_param('preSaleDateTime', pre_sale_date_time)
    add_param('publicVisibilityStartDateTime', public_visibility_start_date_time)

    # Add other filtering parameters
    add_param('marketId', market_id)
    add_param('promoterId', promoter_id)
    add_param('collectionId', collection_id)
    add_param('source', source)
    add_param('domain', domain)

    # Make the API request
    response = requests.get(base_url, params=params)

    # Check if the request was successful
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code}")
        print(response.text)
        return None

# Process the events data into structured format
# Updated Process Events Function with proper end time and duration handling

def process_events(events_data):
    if not events_data or '_embedded' not in events_data:
        print("No events found or invalid response")
        return None

    event_list = []

    for event in events_data['_embedded']['events']:
        # Extract basic event info
        event_info = {
            'id': event['id'],
            'name': event['name'],
            #'url': event['url'],
            'status': event['dates']['status']['code'] if 'status' in event['dates'] else 'unknown',
            'start_date': event['dates']['start'].get('localDate', 'unknown'),
            'start_time': event['dates']['start'].get('localTime', 'unknown'),
        }

        # Add end date and time if available
        if 'end' in event['dates']:
            event_info['end_date'] = event['dates']['end'].get('localDate', 'unknown')
            event_info['end_time'] = event['dates']['end'].get('localTime', 'unknown')
        else:
            event_info['end_date'] = 'not specified'
            event_info['end_time'] = 'not specified'

        # Calculate duration if possible
        if ('end' in event['dates'] and
            event_info['start_date'] != 'unknown' and event_info['start_time'] != 'unknown' and
            event_info['end_date'] != 'unknown' and event_info['end_time'] != 'unknown'):
            try:
                # Fix the datetime format if the time format doesn't include seconds
                start_time = event_info['start_time']
                end_time = event_info['end_time']

                # Add seconds if they're not present
                if len(start_time) == 5:  # Format is HH:MM
                    start_time += ":00"
                if len(end_time) == 5:    # Format is HH:MM
                    end_time += ":00"

                start_dt = datetime.strptime(f"{event_info['start_date']} {start_time}", "%Y-%m-%d %H:%M:%S")
                end_dt = datetime.strptime(f"{event_info['end_date']} {end_time}", "%Y-%m-%d %H:%M:%S")
                duration = end_dt - start_dt
                event_info['duration_minutes'] = round(duration.total_seconds() / 60)
                # Add a human-readable duration
                hours, remainder = divmod(event_info['duration_minutes'], 60)
                if hours > 0:
                    event_info['duration_formatted'] = f"{hours} hour{'s' if hours != 1 else ''}"
                    if remainder > 0:
                        event_info['duration_formatted'] += f" {remainder} minute{'s' if remainder != 1 else ''}"
                else:
                    event_info['duration_formatted'] = f"{remainder} minute{'s' if remainder != 1 else ''}"
            except Exception as e:
                # Handle cases where time formats might be different
                print(f"Error calculating duration for event {event['id']}: {e}")
                event_info['duration_minutes'] = 'unknown'
                event_info['duration_formatted'] = 'unknown'
        else:
            event_info['duration_minutes'] = 'not available'
            event_info['duration_formatted'] = 'not available'

        # Rest of your existing code to extract venue, price, etc.
        # (Remaining code unchanged)

        # Add venue info if available
        if '_embedded' in event and 'venues' in event['_embedded'] and len(event['_embedded']['venues']) > 0:
            venue = event['_embedded']['venues'][0]
            event_info['venue_id'] = venue.get('id', 'unknown')
            event_info['venue_name'] = venue.get('name', 'unknown')
            event_info['venue_city'] = venue.get('city', {}).get('name', 'unknown')
            event_info['venue_state'] = venue.get('state', {}).get('name', 'unknown')
            event_info['venue_country'] = venue.get('country', {}).get('name', 'unknown')
            event_info['venue_postal_code'] = venue.get('postalCode', 'unknown')
            event_info['venue_url'] = venue.get('url', 'unknown')

            # Add venue coordinates if available
            if 'location' in venue:
                event_info['venue_latitude'] = venue['location'].get('latitude', 'unknown')
                event_info['venue_longitude'] = venue['location'].get('longitude', 'unknown')

        # Add price range if available
        if 'priceRanges' in event and len(event['priceRanges']) > 0:
            price_range = event['priceRanges'][0]
            event_info['min_price'] = price_range.get('min', 'unknown')
            event_info['max_price'] = price_range.get('max', 'unknown')
            event_info['currency'] = price_range.get('currency', 'unknown')

        # Add genre/classification if available
        if 'classifications' in event and len(event['classifications']) > 0:
            classification = event['classifications'][0]
            if 'segment' in classification and classification['segment']:
                event_info['segment'] = classification['segment'].get('name', 'unknown')
                event_info['segment_id'] = classification['segment'].get('id', 'unknown')
            if 'genre' in classification and classification['genre']:
                event_info['genre'] = classification['genre'].get('name', 'unknown')
                event_info['genre_id'] = classification['genre'].get('id', 'unknown')
            if 'subGenre' in classification and classification['subGenre']:
                event_info['subgenre'] = classification['subGenre'].get('name', 'unknown')
                event_info['subgenre_id'] = classification['subGenre'].get('id', 'unknown')
            if 'type' in classification and classification['type']:
                event_info['type'] = classification['type'].get('name', 'unknown')
                event_info['type_id'] = classification['type'].get('id', 'unknown')
            if 'subType' in classification and classification['subType']:
                event_info['subtype'] = classification['subType'].get('name', 'unknown')
                event_info['subtype_id'] = classification['subType'].get('id', 'unknown')

        # Add image URLs if available
        if 'images' in event and len(event['images']) > 0:
            # Find standard image
            standard_images = [img for img in event['images'] if img.get('ratio') == '16_9' and img.get('width') > 500]
            if standard_images:
                event_info['image_url'] = standard_images[0]['url']
            else:
                event_info['image_url'] = event['images'][0]['url']  # Fallback to first image

        # Add attractions/performers if available
        if '_embedded' in event and 'attractions' in event['_embedded'] and len(event['_embedded']['attractions']) > 0:
            attractions = []
            attraction_ids = []
            for attraction in event['_embedded']['attractions']:
                attractions.append(attraction.get('name', 'unknown'))
                attraction_ids.append(attraction.get('id', 'unknown'))
            event_info['attractions'] = ', '.join(attractions)
            event_info['attraction_ids'] = ', '.join(attraction_ids)

        # Add ticket info if available
        if 'sales' in event and 'public' in event['sales']:
            sales_public = event['sales']['public']
            if 'startDateTime' in sales_public:
                event_info['on_sale_date'] = sales_public['startDateTime']
            if 'endDateTime' in sales_public:
                event_info['off_sale_date'] = sales_public['endDateTime']

        # Add pre-sale info if available
        if 'sales' in event and 'presales' in event['sales'] and len(event['sales']['presales']) > 0:
            presale = event['sales']['presales'][0]  # Get the first pre-sale info
            event_info['presale_name'] = presale.get('name', 'unknown')
            event_info['presale_start_date'] = presale.get('startDateTime', 'unknown')
            event_info['presale_end_date'] = presale.get('endDateTime', 'unknown')

        event_list.append(event_info)

    # Create DataFrame
    df = pd.DataFrame(event_list)
    return df

# Modified display function to show end time and duration in the output
def display_structured_events(events_data):
    # Process the events
    events_df = process_events(events_data)

    if events_df is not None and not events_df.empty:
        # Display info
        print(f"Found {len(events_df)} events")

        # Display pagination info if available
        if 'pagination_info' in events_data:
            pagination = events_data['pagination_info']
            print("\n=== PAGINATION SUMMARY ===")
            print(f"Events per page: {pagination.get('events_per_page', 'Unknown')}")
            print(f"Total events available: {pagination.get('total_events', 'Unknown')}")
            print(f"Total pages available: {pagination.get('total_pages', 'Unknown')}")
            print("===========================")

        # Show a sample with focus on duration and end time
        print("\n=== SAMPLE EVENT WITH TIME INFO ===")
        sample = events_df.head(1)
        time_columns = ['name', 'start_date', 'start_time', 'end_date', 'end_time', 'duration_formatted']
        display(sample[time_columns])

        # Display as styled table (uncomment if needed)
        # print("\n=== EVENTS TABLE ===")
        # display(events_df)

        # Convert to JSON format and display
        print("\n=== EVENTS JSON ===")
        events_json = events_df.to_dict(orient='records')
        print(json.dumps(events_json, indent=2))

        # Create results dictionary including pagination info if available
        results = {
            'dataframe': events_df,
            'json': events_json
        }

        if 'pagination_info' in events_data:
            results['pagination'] = events_data['pagination_info']

        return results
    else:
        print("No events to display")
        return None

# Function to fetch events with pagination
def fetch_all_events(max_pages=5, events_per_page=20, **search_params):
    all_events = []
    total_elements = 0
    total_pages = 0
    pagination_info = {}

    for page in range(max_pages):
        # Update the page parameter
        search_params['page'] = page
        search_params['size'] = events_per_page

        # Fetch the current page
        events_data = fetch_events(**search_params)

        # Check if we have a valid response
        if events_data and '_embedded' in events_data and 'events' in events_data['_embedded']:
            # Get page info
            page_info = events_data.get('page', {})
            current_page = page_info.get('number', page)
            events_per_page_actual = page_info.get('size', events_per_page)
            total_elements = page_info.get('totalElements', 0)
            total_pages = page_info.get('totalPages', 0)

            # Store pagination information (from the first page)
            if page == 0:
                pagination_info = {
                    'events_per_page': events_per_page_actual,
                    'total_events': total_elements,
                    'total_pages': total_pages
                }

                # Print pagination information
                print(f"\n=== PAGINATION INFORMATION ===")
                print(f"Events per page: {events_per_page_actual}")
                print(f"Total events available: {total_elements}")
                print(f"Total pages available: {total_pages}")
                print(f"Maximum pages to fetch: {min(max_pages, total_pages)}")
                print("===============================\n")

            # Process the events and add them to our collection
            page_events = events_data['_embedded']['events']
            all_events.extend(page_events)

            print(f"Fetched page {current_page+1}/{total_pages} ({len(page_events)} events)")

            # Check if we've reached the last page
            if page >= total_pages - 1:
                break
        else:
            print(f"No events found on page {page+1} or error in response")
            break

    # Convert all events to raw format for processing
    raw_events_data = {
        '_embedded': {
            'events': all_events
        },
        'pagination_info': pagination_info
    }

    print(f"Total events fetched: {len(all_events)} out of {total_elements}")
    return raw_events_data

# Function to display results as table and JSON


# Save results to CSV, JSON, or Excel
def save_results(results, prefix='events'):
    if results is None or 'json' not in results:
        print("No JSON results to save")
        return

    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

    # Specify an absolute path where you want to save the file
    # Change this to a directory you have write access to
    save_directory = "/path/to/your/directory"  # Example: "C:/Users/YourName/Documents" on Windows or "/home/username/Documents" on Linux

    filename = f"{save_directory}/{prefix}_{timestamp}.json"

    try:
        with open(filename, 'w') as f:
            json.dump(results['json'], f, indent=2)
        print(f"Successfully saved JSON to {filename}")
    except Exception as e:
        print(f"Error saving file: {e}")

    print(f"Saved JSON to {filename}")
# Example usage
def main():
    # Define your search parameters with more options
    params = {
        'country_code': "US",
        'classification_name': "music",  # Music, Sports, Arts, Film, etc.
        'city': "New York",
        'state_code': "NY",
        'start_date_time': "2025-05-15T00:00:00Z",  # Events after this date
        #'end_date_time': "2025-08-15T23:59:59Z",    # Events before this date
        'sort': "date,asc",                        # Sort by date ascending
        #'include_family': "yes"                    # Include family-friendly events
    }

    # Define pagination parameters
    max_pages = 10     # Maximum number of pages to fetch
    events_per_page = 200  # Number of events per page to request

    # You can control the number of events per page by changing the 'size' parameter
    # The API default is 20, but you can set it up to the API's maximum (usually 50-100)
    params['size'] = 200 # Uncomment to override events_per_page

    # Fetch events with pagination
    #print(f"Fetching {params['classification_name']} events in {params['city']}, {params['state_code']}...")
    #print(f"Time range: {params['start_date_time']} to {params['end_date_time']}")
    #print(f"Pagination settings: max {max_pages} pages, {events_per_page} events per page")

    events_data = fetch_all_events(max_pages=max_pages, events_per_page=events_per_page, **params)

    # Display structured results
    results = display_structured_events(events_data)

    # Save results to files
    save_results(results, prefix='_events')

    # Return all the information
    if results and 'pagination' in results:
        print("\n=== PAGINATION SUMMARY ===")
        print(f"Events per page: {results['pagination']['events_per_page']}")
        print(f"Total events available: {results['pagination']['total_events']}")
        print(f"Total pages available: {results['pagination']['total_pages']}")

    return results

# Run the main function
if __name__ == "__main__":
    results = main()

# **classification endpoint **

In [ ]:
import requests
import json
import pandas as pd  # Added for Colab display
from google.colab import files  # For downloading files in Colab
from google.colab import userdata  # For accessing secrets

# Get API key from Colab secrets
try:
    API_KEY = userdata.get('TICKETMASTER_API_KEY')
    print("API key loaded from Colab secrets")
except Exception as e:
    print(f"Error loading API key from secrets: {e}")

# Endpoints
CLASSIFICATIONS_URL = "https://app.ticketmaster.com/discovery/v2/classifications.json"
PAGE_SIZE = 500

def fetch_all_classifications():
    page = 0
    all_classifications = []

    while True:
        params = {
            "apikey": API_KEY,
            "size": PAGE_SIZE,
            "page": page
        }

        print(f"Fetching page {page}...")
        response = requests.get(CLASSIFICATIONS_URL, params=params)
        response.raise_for_status()
        data = response.json()

        embedded = data.get("_embedded", {})
        classifications = embedded.get("classifications", [])

        if not classifications:
            break

        all_classifications.extend(classifications)

        page_info = data.get("page", {})
        total_pages = page_info.get("totalPages", 0)
        print(f"Fetched page {page + 1} of {total_pages}")

        page += 1
        if page >= total_pages:
            break

    return all_classifications

def build_structured_entries(classifications):
    entries = []

    for c in classifications:
        classification_id = c.get("id")
        classification_name = c.get("name")
        classification_primary = c.get("primary", False)  # Added primary flag
        classification_type = c.get("type")  # Added classification type

        # Get segment information
        segment = c.get("segment", {})
        if segment:
            segment_name = segment.get("name")
            segment_id = segment.get("id")
            # Added more segment details
            segment_locale = segment.get("locale")
            segment_primary = segment.get("primary", False)
            segment_level = segment.get("level")

            genres = segment.get("_embedded", {}).get("genres", [])

            if not genres:
                entries.append({
                    "classificationId": classification_id,
                    "classificationName": classification_name,
                    "classificationType": classification_type,
                    "classificationPrimary": classification_primary,
                    "segment": segment_name,
                    "segmentId": segment_id,
                    "segmentLocale": segment_locale,
                    "segmentPrimary": segment_primary,
                    "segmentLevel": segment_level,
                    "genre": None,
                    "genreId": None,
                    "genreLocale": None,
                    "genrePrimary": None,
                    "genreLevel": None,
                    "subgenre": None,
                    "subgenreId": None,
                    "subgenreLocale": None,
                    "subgenrePrimary": None,
                    "subgenreLevel": None
                })
                continue

            for genre in genres:
                genre_name = genre.get("name")
                genre_id = genre.get("id")
                # Added more genre details
                genre_locale = genre.get("locale")
                genre_primary = genre.get("primary", False)
                genre_level = genre.get("level")

                subgenres = genre.get("_embedded", {}).get("subgenres", [])

                if not subgenres:
                    entries.append({
                        "classificationId": classification_id,
                        "classificationName": classification_name,
                        "classificationType": classification_type,
                        "classificationPrimary": classification_primary,
                        "segment": segment_name,
                        "segmentId": segment_id,
                        "segmentLocale": segment_locale,
                        "segmentPrimary": segment_primary,
                        "segmentLevel": segment_level,
                        "genre": genre_name,
                        "genreId": genre_id,
                        "genreLocale": genre_locale,
                        "genrePrimary": genre_primary,
                        "genreLevel": genre_level,
                        "subgenre": None,
                        "subgenreId": None,
                        "subgenreLocale": None,
                        "subgenrePrimary": None,
                        "subgenreLevel": None
                    })
                else:
                    for subgenre in subgenres:
                        # Added more subgenre details
                        subgenre_locale = subgenre.get("locale")
                        subgenre_primary = subgenre.get("primary", False)
                        subgenre_level = subgenre.get("level")

                        entries.append({
                            "classificationId": classification_id,
                            "classificationName": classification_name,
                            "classificationType": classification_type,
                            "classificationPrimary": classification_primary,
                            "segment": segment_name,
                            "segmentId": segment_id,
                            "segmentLocale": segment_locale,
                            "segmentPrimary": segment_primary,
                            "segmentLevel": segment_level,
                            "genre": genre_name,
                            "genreId": genre_id,
                            "genreLocale": genre_locale,
                            "genrePrimary": genre_primary,
                            "genreLevel": genre_level,
                            "subgenre": subgenre.get("name"),
                            "subgenreId": subgenre.get("id"),
                            "subgenreLocale": subgenre_locale,
                            "subgenrePrimary": subgenre_primary,
                            "subgenreLevel": subgenre_level
                        })

    return entries

# Main execution for Colab
if __name__ == "__main__":

    # Fetch all classifications
    print("Fetching classifications from Ticketmaster API...")
    classifications = fetch_all_classifications()

    # Save raw classifications to file
    with open("raw_classifications.json", "w") as f:
        json.dump(classifications, f, indent=2)
    print("Saved raw classifications to raw_classifications.json")

    # Build structured entries
    print("Processing classifications into structured format...")
    structured_entries = build_structured_entries(classifications)

    # Save structured data to file
    with open("full_classifications.json", "w") as f:
        json.dump(structured_entries, f, indent=2)
    print("Saved structured classifications to full_classifications.json")

    # Create a dataframe for display in Colab
    df = pd.DataFrame(structured_entries)
    print(f"\nCreated dataframe with {len(df)} rows and {len(df.columns)} columns")
    print("\nPreview of the data:")
    display(df.head())

    # Download files in Colab
    print("\nDownloading files to your computer...")
    files.download("raw_classifications.json")
    files.download("full_classifications.json")

    print("\nProcess completed successfully!")

API key loaded from Colab secrets
Fetching classifications from Ticketmaster API...
Fetching page 0...
Fetched page 1 of 1
Saved raw classifications to raw_classifications.json
Processing classifications into structured format...
Saved structured classifications to full_classifications.json

Created dataframe with 1086 rows and 19 columns

Preview of the data:


,classificationId,classificationName,classificationType,classificationPrimary,segment,segmentId,segmentLocale,segmentPrimary,segmentLevel,genre,genreId,genreLocale,genrePrimary,genreLevel,subgenre,subgenreId,subgenreLocale,subgenrePrimary,subgenreLevel
0,None,None,None,False,Miscellaneous,KZFzniwnSyZfZ7v7n1,en-us,False,None,Casino/Gaming,KnvZfZ7vAAa,en-us,False,None,Casino/Gaming,KZazBEonSMnZfZ7vFnt,en-us,False,None
1,None,None,None,False,Miscellaneous,KZFzniwnSyZfZ7v7n1,en-us,False,None,Comedy,KnvZfZ7vAA1,en-us,False,None,Comedy,KZazBEonSMnZfZ7vFnn,en-us,False,None
2,None,None,None,False,Miscellaneous,KZFzniwnSyZfZ7v7n1,en-us,False,None,Community/Civic,KnvZfZ7vAAE,en-us,False,None,Community/Civic,KZazBEonSMnZfZ7vFlv,en-us,False,None
3,None,None,None,False,Miscellaneous,KZFzniwnSyZfZ7v7n1,en-us,False,None,Community/Cultural,KnvZfZ7v7lE,en-us,False,None,Public Skating,KZazBEonSMnZfZ7vAvk,en-us,False,None
4,None,None,None,False,Miscellaneous,KZFzniwnSyZfZ7v7n1,en-us,False,None,Fairs & Festivals,KnvZfZ7vAeE,en-us,False,None,Fairs & Festivals,KZazBEonSMnZfZ7vF1F,en-us,False,None


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


Process completed successfully!


fetch genre/sub-genre... using segment id

In [ ]:
import requests
import json
import pandas as pd
import time
from google.colab import files
from google.colab import userdata
from IPython.display import display, HTML

# Get API key from Colab secrets
try:
    API_KEY = userdata.get('TICKETMASTER_API_KEY')
    print("API key loaded from Colab secrets")
except Exception as e:
    print(f"Error loading API key from secrets: {e}")
    API_KEY = ""  # Will be requested later if empty

# Ticketmaster API endpoints
CLASSIFICATIONS_URL = "https://app.ticketmaster.com/discovery/v2/classifications.json"
EVENTS_URL = "https://app.ticketmaster.com/discovery/v2/events.json"
PAGE_SIZE = 200  # Maximum allowed by API

class TicketmasterAPI:
    def __init__(self, api_key):
        self.api_key = api_key
        # Dictionary to cache classification data
        self.classifications_cache = None
        # Dictionary for mapping IDs to names for easier lookup
        self.id_name_mapping = {
            'segments': {},
            'genres': {},
            'subgenres': {}
        }

    def _handle_rate_limit(self, response):
        """Handle rate limiting from Ticketmaster API"""
        if response.status_code == 429:  # Too Many Requests
            retry_after = int(response.headers.get('Retry-After', 1))
            print(f"Rate limited. Waiting {retry_after} seconds...")
            time.sleep(retry_after)
            return True
        return False

    def fetch_all_classifications(self):
        """Fetch all classifications from Ticketmaster API"""
        if self.classifications_cache is not None:
            print("Using cached classifications data")
            return self.classifications_cache

        page = 0
        all_classifications = []

        while True:
            params = {
                "apikey": self.api_key,
                "size": PAGE_SIZE,
                "page": page
            }

            print(f"Fetching classifications page {page}...")
            response = requests.get(CLASSIFICATIONS_URL, params=params)

            # Handle rate limiting
            if self._handle_rate_limit(response):
                continue

            response.raise_for_status()
            data = response.json()

            embedded = data.get("_embedded", {})
            classifications = embedded.get("classifications", [])

            if not classifications:
                break

            all_classifications.extend(classifications)

            # Update our ID to name mappings for easier lookup
            self._update_id_mappings(classifications)

            page_info = data.get("page", {})
            total_pages = page_info.get("totalPages", 0)
            print(f"Fetched classifications page {page + 1} of {total_pages}")

            page += 1
            if page >= total_pages:
                break

        self.classifications_cache = all_classifications
        return all_classifications

    def _update_id_mappings(self, classifications):
        """Update ID to name mappings for easier lookup"""
        for c in classifications:
            segment = c.get("segment", {})
            if segment:
                segment_id = segment.get("id")
                segment_name = segment.get("name")
                if segment_id and segment_name:
                    self.id_name_mapping['segments'][segment_id] = segment_name

                genres = segment.get("_embedded", {}).get("genres", [])
                for genre in genres:
                    genre_id = genre.get("id")
                    genre_name = genre.get("name")
                    if genre_id and genre_name:
                        self.id_name_mapping['genres'][genre_id] = genre_name

                    subgenres = genre.get("_embedded", {}).get("subgenres", [])
                    for subgenre in subgenres:
                        subgenre_id = subgenre.get("id")
                        subgenre_name = subgenre.get("name")
                        if subgenre_id and subgenre_name:
                            self.id_name_mapping['subgenres'][subgenre_id] = subgenre_name

    def get_id_name_mapping(self):
        """Get ID to name mappings for segments, genres, and subgenres"""
        # Ensure classifications have been fetched
        if not self.classifications_cache:
            self.fetch_all_classifications()
        return self.id_name_mapping

    def fetch_events_by_classification(self, classification_type, classification_id, country_code=None, city=None, start_date=None, end_date=None, size=50, page=0, sort=None):
        """
        Fetch events by classification type and ID

        Parameters:
        - classification_type: 'segment', 'genre', or 'subgenre'
        - classification_id: ID of the classification
        - country_code: Optional country code filter (e.g., 'US')
        - city: Optional city name filter
        - start_date: Optional start date in ISO format (e.g., '2023-05-01T00:00:00Z')
        - end_date: Optional end date in ISO format (e.g., '2023-06-01T00:00:00Z')
        - size: Number of events per page (max 200)
        - page: Page number
        - sort: Sort order (e.g., 'date,asc', 'relevance,desc')

        Returns:
        - Dictionary of events data
        """
        # Validate classification type
        if classification_type not in ['segment', 'genre', 'subgenre']:
            raise ValueError("classification_type must be 'segment', 'genre', or 'subgenre'")

        # Build parameters
        params = {
            "apikey": self.api_key,
            f"{classification_type}Id": classification_id,
            "size": min(size, PAGE_SIZE),
            "page": page
        }

        # Add optional parameters if provided
        if country_code:
            params["countryCode"] = country_code
        if city:
            params["city"] = city
        if start_date:
            params["startDateTime"] = start_date
        if end_date:
            params["endDateTime"] = end_date
        if sort:
            params["sort"] = sort

        # Fetch events
        print(f"Fetching events for {classification_type} ID {classification_id}...")
        response = requests.get(EVENTS_URL, params=params)

        # Handle rate limiting
        if self._handle_rate_limit(response):
            return self.fetch_events_by_classification(
                classification_type, classification_id, country_code, city,
                start_date, end_date, size, page, sort
            )

        response.raise_for_status()
        return response.json()

    def fetch_all_events_by_classification(self, classification_type, classification_id, country_code=None, city=None, start_date=None, end_date=None, max_pages=10, sort=None):
        """
        Fetch all events by classification type and ID with pagination

        Parameters:
        - See fetch_events_by_classification
        - max_pages: Maximum number of pages to fetch (to avoid excessive API calls)

        Returns:
        - List of event objects
        """
        all_events = []
        page = 0

        while page < max_pages:
            data = self.fetch_events_by_classification(
                classification_type, classification_id, country_code, city,
                start_date, end_date, PAGE_SIZE, page, sort
            )

            embedded = data.get("_embedded", {})
            events = embedded.get("events", [])

            if not events:
                break

            all_events.extend(events)

            page_info = data.get("page", {})
            total_pages = page_info.get("totalPages", 0)
            current_page = page_info.get("number", 0)

            print(f"Fetched events page {current_page + 1} of {total_pages} ({len(events)} events)")

            page += 1
            if page >= total_pages:
                break

        return all_events

    def process_events_data(self, events):
        """
        Process events data into a structured format

        Parameters:
        - events: List of event objects

        Returns:
        - List of dictionaries with structured event data
        """
        processed_events = []

        for event in events:
            event_data = {
                "id": event.get("id"),
                "name": event.get("name"),
                "url": event.get("url"),
                "status": event.get("dates", {}).get("status", {}).get("code"),
                "startDate": event.get("dates", {}).get("start", {}).get("dateTime"),
                "timezone": event.get("dates", {}).get("timezone"),
                "images": [img.get("url") for img in event.get("images", [])[:1]],  # Just get the first image URL
                "priceRanges": event.get("priceRanges", []),
                "seatmap": event.get("seatmap", {}).get("staticUrl"),
                "ticketLimit": event.get("ticketLimit", {}),
            }

            # Add venue info if available
            embedded = event.get("_embedded", {})
            venues = embedded.get("venues", [])
            if venues:
                venue = venues[0]  # Get the first venue
                event_data["venue"] = {
                    "id": venue.get("id"),
                    "name": venue.get("name"),
                    "city": venue.get("city", {}).get("name"),
                    "state": venue.get("state", {}).get("name"),
                    "country": venue.get("country", {}).get("name"),
                    "address": venue.get("address", {}).get("line1"),
                    "location": {
                        "latitude": venue.get("location", {}).get("latitude"),
                        "longitude": venue.get("location", {}).get("longitude")
                    }
                }

            # Add attractions info if available
            attractions = embedded.get("attractions", [])
            if attractions:
                event_data["attractions"] = [{
                    "id": attraction.get("id"),
                    "name": attraction.get("name"),
                    "type": attraction.get("type"),
                    "url": attraction.get("url")
                } for attraction in attractions]

            # Add classifications info
            classifications = event.get("classifications", [])
            if classifications:
                classification = classifications[0]  # Get the first classification
                event_data["classification"] = {
                    "segment": {
                        "id": classification.get("segment", {}).get("id"),
                        "name": classification.get("segment", {}).get("name")
                    },
                    "genre": {
                        "id": classification.get("genre", {}).get("id"),
                        "name": classification.get("genre", {}).get("name")
                    },
                    "subgenre": {
                        "id": classification.get("subGenre", {}).get("id"),
                        "name": classification.get("subGenre", {}).get("name")
                    }
                }

            processed_events.append(event_data)

        return processed_events

    def display_classification_selector(self):
        """Display interactive classification selector in Colab"""
        # Ensure classifications have been fetched
        if not self.classifications_cache:
            self.fetch_all_classifications()

        # Prepare data for display
        segments = [{"id": k, "name": v} for k, v in self.id_name_mapping['segments'].items()]
        genres = [{"id": k, "name": v} for k, v in self.id_name_mapping['genres'].items()]
        subgenres = [{"id": k, "name": v} for k, v in self.id_name_mapping['subgenres'].items()]

        # Sort by name
        segments.sort(key=lambda x: x['name'])
        genres.sort(key=lambda x: x['name'])
        subgenres.sort(key=lambda x: x['name'])

        # Display segment selector
        print("\n=== SEGMENTS ===")
        segment_df = pd.DataFrame(segments)
        display(segment_df)

        # Display genre selector
        print("\n=== GENRES ===")
        genre_df = pd.DataFrame(genres)
        display(genre_df)

        # Display subgenre selector
        print("\n=== SUBGENRES ===")
        subgenre_df = pd.DataFrame(subgenres)
        display(subgenre_df)

        return {
            'segments': segment_df,
            'genres': genre_df,
            'subgenres': subgenre_df
        }

def example_usage():
    """Example usage of the Ticketmaster API wrapper"""
    # Make sure you have your API key set
    global API_KEY
    if not API_KEY:
        print("API key not found in Colab secrets.")
        print("To add your API key to Colab secrets:")
        print("1. Click on the 🔑 icon in the left sidebar")
        print("2. Add a new secret with name 'TICKETMASTER_API_KEY' and your API key as the value")
        print("3. Restart the runtime and run again")
        API_KEY = input("Or enter your Ticketmaster API key manually: ")

    # Initialize the API wrapper
    tm_api = TicketmasterAPI(API_KEY)

    # Fetch classifications to populate the mapping
    tm_api.fetch_all_classifications()

    # Display classification selector
    print("\nClassification Selector:")
    selector_dfs = tm_api.display_classification_selector()

    # Example: Fetch events by segment ID
    # Music segment ID: KZFzniwnSyZfZ7v7nJ
    segment_id = input("\nEnter segment ID (or press Enter to skip): ").strip()
    if segment_id:
        segment_name = tm_api.id_name_mapping['segments'].get(segment_id, "Unknown")
        print(f"\nFetching events for segment: {segment_name} (ID: {segment_id})")
        segment_events = tm_api.fetch_all_events_by_classification('segment', segment_id, max_pages=2)
        processed_segment_events = tm_api.process_events_data(segment_events)

        # Save segment events to file
        with open(f"segment_{segment_id}_events.json", "w") as f:
            json.dump(processed_segment_events, f, indent=2)
        print(f"Saved {len(processed_segment_events)} segment events to segment_{segment_id}_events.json")

        # Display preview
        segment_events_df = pd.DataFrame(processed_segment_events)
        if not segment_events_df.empty:
            display(segment_events_df[['name', 'startDate', 'venue']].head())
            files.download(f"segment_{segment_id}_events.json")

    # Example: Fetch events by genre ID
    genre_id = input("\nEnter genre ID (or press Enter to skip): ").strip()
    if genre_id:
        genre_name = tm_api.id_name_mapping['genres'].get(genre_id, "Unknown")
        print(f"\nFetching events for genre: {genre_name} (ID: {genre_id})")
        genre_events = tm_api.fetch_all_events_by_classification('genre', genre_id, max_pages=2)
        processed_genre_events = tm_api.process_events_data(genre_events)

        # Save genre events to file
        with open(f"genre_{genre_id}_events.json", "w") as f:
            json.dump(processed_genre_events, f, indent=2)
        print(f"Saved {len(processed_genre_events)} genre events to genre_{genre_id}_events.json")

        # Display preview
        genre_events_df = pd.DataFrame(processed_genre_events)
        if not genre_events_df.empty:
            display(genre_events_df[['name', 'startDate', 'venue']].head())
            files.download(f"genre_{genre_id}_events.json")

    # Example: Fetch events by subgenre ID
    subgenre_id = input("\nEnter subgenre ID (or press Enter to skip): ").strip()
    if subgenre_id:
        subgenre_name = tm_api.id_name_mapping['subgenres'].get(subgenre_id, "Unknown")
        print(f"\nFetching events for subgenre: {subgenre_name} (ID: {subgenre_id})")
        subgenre_events = tm_api.fetch_all_events_by_classification('subgenre', subgenre_id, max_pages=2)
        processed_subgenre_events = tm_api.process_events_data(subgenre_events)

        # Save subgenre events to file
        with open(f"subgenre_{subgenre_id}_events.json", "w") as f:
            json.dump(processed_subgenre_events, f, indent=2)
        print(f"Saved {len(processed_subgenre_events)} subgenre events to subgenre_{subgenre_id}_events.json")

        # Display preview
        subgenre_events_df = pd.DataFrame(processed_subgenre_events)
        if not subgenre_events_df.empty:
            display(subgenre_events_df[['name', 'startDate', 'venue']].head())
            files.download(f"subgenre_{subgenre_id}_events.json")

if __name__ == "__main__":
    example_usage()

API key loaded from Colab secrets
Fetching classifications page 0...
Fetched classifications page 1 of 1

Classification Selector:

=== SEGMENTS ===


,id,name
0,KZFzniwnSyZfZ7v7na,Arts & Theatre
1,KZFzniwnSyZfZ7v7nn,Film
2,KZFzniwnSyZfZ7v7n1,Miscellaneous
3,KZFzniwnSyZfZ7v7nJ,Music
4,KZFzniwnSyZfZ7v7nE,Sports
5,KZFzniwnSyZfZ7v7nl,Undefined



=== GENRES ===


,id,name
0,KnvZfZ7vAke,Action/Adventure
1,KnvZfZ7vAvv,Alternative
2,KnvZfZ7vAkd,Animation
3,KnvZfZ7vAeI,Aquatics
4,KnvZfZ7vAk7,Arthouse
...,...,...
119,KnvZfZ7vAA7,Volleyball
120,KnvZfZ7vAAA,Waterpolo
121,KnvZfZ7vAeF,World
122,KnvZfZ7vAAk,Wrestling



=== SUBGENRES ===


...

<IPython.core.display.Javascript object>


Enter subgenre ID (or press Enter to skip): KZazBEonSMnZfZ7vAnA

Fetching events for subgenre: A Cappella (ID: KZazBEonSMnZfZ7vAnA)
Fetching events for subgenre ID KZazBEonSMnZfZ7vAnA...
Fetched events page 1 of 1351 (200 events)
Fetching events for subgenre ID KZazBEonSMnZfZ7vAnA...
Fetched events page 2 of 1351 (200 events)
Saved 400 subgenre events to subgenre_KZazBEonSMnZfZ7vAnA_events.json


,name,startDate,venue
0,New York Yankees vs. Baltimore Orioles,2025-06-21T17:05:00Z,"{'id': 'KovZpZA6t77A', 'name': 'Yankee Stadium..."
1,New York Yankees vs. Houston Astros,2025-08-08T23:05:00Z,"{'id': 'KovZpZA6t77A', 'name': 'Yankee Stadium..."
2,New York Yankees vs. Athletics,2025-06-28T17:05:00Z,"{'id': 'KovZpZA6t77A', 'name': 'Yankee Stadium..."
3,New York Yankees vs. New York Mets,2025-05-16T23:05:00Z,"{'id': 'KovZpZA6t77A', 'name': 'Yankee Stadium..."
4,New York Yankees vs. Baltimore Orioles,2025-09-27T17:05:00Z,"{'id': 'KovZpZA6t77A', 'name': 'Yankee Stadium..."


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# fetch events - **events endpoint**

In [ ]:
from google.colab import userdata

import requests
import pandas as pd
import json
from datetime import datetime
from IPython.display import display, HTML

api_key = userdata.get('TICKETMASTER_API_KEY')
base_url = "https://app.ticketmaster.com/discovery/v2/events.json"

def fetch_events(country_code="US",
                 classification_name=None,
                 city=None,
                 state_code=None,
                 start_date=None,
                 keyword=None,
                 size=20,
                 page=0,
                 sort="date,asc"):

    # Define parameters
    params = {
        'apikey': api_key,
        'countryCode': country_code,
        'size': size,
        'page': page,
        'sort': sort
    }

    # Add optional parameters if provided
    if classification_name:
        params['classificationName'] = classification_name
    if city:
        params['city'] = city
    if state_code:
        params['stateCode'] = state_code
    if start_date:
        params['startDateTime'] = start_date
    if keyword:
        params['keyword'] = keyword

    # Make the API request
    response = requests.get(base_url, params=params)

    # Check if the request was successful
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code}")
        print(response.text)
        return None

# Process the events data into structured format
def process_events(events_data):
    if not events_data or '_embedded' not in events_data:
        print("No events found or invalid response")
        return None

    event_list = []

    for event in events_data['_embedded']['events']:
        # Extract basic event info
        event_info = {
            'id': event['id'],
            'name': event['name'],
            'url': event['url'],
            'status': event['dates']['status']['code'] if 'status' in event['dates'] else 'unknown',
            'start_date': event['dates']['start'].get('localDate', 'unknown'),
            'start_time': event['dates']['start'].get('localTime', 'unknown'),
        }

        # Add venue info if available
        if '_embedded' in event and 'venues' in event['_embedded'] and len(event['_embedded']['venues']) > 0:
            venue = event['_embedded']['venues'][0]
            event_info['venue_name'] = venue.get('name', 'unknown')
            event_info['venue_city'] = venue.get('city', {}).get('name', 'unknown')
            event_info['venue_state'] = venue.get('state', {}).get('name', 'unknown')
            event_info['venue_country'] = venue.get('country', {}).get('name', 'unknown')

            # Add venue coordinates if available
            if 'location' in venue:
                event_info['venue_latitude'] = venue['location'].get('latitude', 'unknown')
                event_info['venue_longitude'] = venue['location'].get('longitude', 'unknown')

        # Add price range if available
        if 'priceRanges' in event and len(event['priceRanges']) > 0:
            price_range = event['priceRanges'][0]
            event_info['min_price'] = price_range.get('min', 'unknown')
            event_info['max_price'] = price_range.get('max', 'unknown')
            event_info['currency'] = price_range.get('currency', 'unknown')

        # Add genre/classification if available
        if 'classifications' in event and len(event['classifications']) > 0:
            classification = event['classifications'][0]
            if 'segment' in classification and classification['segment']:
                event_info['segment'] = classification['segment'].get('name', 'unknown')
            if 'genre' in classification and classification['genre']:
                event_info['genre'] = classification['genre'].get('name', 'unknown')
            if 'subGenre' in classification and classification['subGenre']:
                event_info['subgenre'] = classification['subGenre'].get('name', 'unknown')

        # Add image URLs if available
        if 'images' in event and len(event['images']) > 0:
            # Find standard image
            standard_images = [img for img in event['images'] if img.get('ratio') == '16_9' and img.get('width') > 500]
            if standard_images:
                event_info['image_url'] = standard_images[0]['url']
            else:
                event_info['image_url'] = event['images'][0]['url']  # Fallback to first image

        # Add attractions/performers if available
        if '_embedded' in event and 'attractions' in event['_embedded'] and len(event['_embedded']['attractions']) > 0:
            attractions = []
            for attraction in event['_embedded']['attractions']:
                attractions.append(attraction.get('name', 'unknown'))
            event_info['attractions'] = ', '.join(attractions)

        # Add ticket info if available
        if 'sales' in event and 'public' in event['sales']:
            sales_public = event['sales']['public']
            if 'startDateTime' in sales_public:
                event_info['on_sale_date'] = sales_public['startDateTime']
            if 'endDateTime' in sales_public:
                event_info['off_sale_date'] = sales_public['endDateTime']

        event_list.append(event_info)

    # Create DataFrame
    df = pd.DataFrame(event_list)
    return df

# Function to display results as table and JSON
def display_structured_events(events_data):
    # Process the events
    events_df = process_events(events_data)

    if events_df is not None and not events_df.empty:
        # Display info
        print(f"Found {len(events_df)} events")

        # Display as styled table
        print("\n=== EVENTS TABLE ===")
        display(events_df)

        # Convert to JSON format and display
        print("\n=== EVENTS JSON ===")
        events_json = events_df.to_dict(orient='records')
        print(json.dumps(events_json, indent=2))

        return {
            'dataframe': events_df,
            'json': events_json
        }
    else:
        print("No events to display")
        return None

# Example usage
def main():
    # Define your search parameters
    params = {
        'country_code': "US",
        'classification_name': "sports",  # Or Could be Music, Sports, Arts, Film, etc.
        'city': "Los Angeles",
        'state_code': "CA",
        'size': 10  # Number of results
    }

    # Fetch events
    print(f"Fetching {params['classification_name']} events in {params['city']}, {params['state_code']}...")
    events_data = fetch_events(**params)

    # Display structured results
    results = display_structured_events(events_data)

    # Optionally save results
    # save_results(results)

    return results

# Run the main function
results = main()

# events endpoint

In [ ]:
from google.colab import userdata

import requests
import pandas as pd
import json
from datetime import datetime
from IPython.display import display, HTML

api_key = userdata.get('TICKETMASTER_API_KEY')
base_url = "https://app.ticketmaster.com/discovery/v2/events.json"

def fetch_events(
    # Basic parameters
    country_code=None,
    size=100,
    page=0,
    sort="date,asc",

    # Identification parameters
    id=None,
    keyword=None,
    attraction_id=None,
    venue_id=None,

    # Geographic parameters
    city=None,
    state_code=None,
    postal_code=None,
    latlong=None,
    geo_point=None,
    radius=None,
    unit="miles",
    dma_id=None,

    # Date and time parameters
    start_date_time=None,
    end_date_time=None,
    local_start_date_time=None,
    local_start_end_date_time=None,
    start_end_date_time=None,

    # Classification parameters
    classification_name=None,
    classification_id=None,
    segment_id=None,
    segment_name=None,
    genre_id=None,
    sub_genre_id=None,
    type_id=None,
    sub_type_id=None,

    # Sale parameters
    onsale_start_date_time=None,
    onsale_end_date_time=None,
    onsale_on_start_date=None,
    onsale_on_after_start_date=None,
    pre_sale_date_time=None,
    public_visibility_start_date_time=None,

    # Include/exclude parameters
    include_tba="yes",
    include_tbd="yes",
    include_test="no",
    include_family="yes",
    include_spellcheck="no",

    # Other filtering parameters
    market_id=None,
    promoter_id=None,
    collection_id=None,
    source=None,
    locale="en",
    domain=None,
    preferred_country="us"
):
    # Define parameters dict with base parameters
    params = {
        'apikey': api_key,
        'size': size,
        'page': page,
        'sort': sort,
        'includeTBA': include_tba,
        'includeTBD': include_tbd,
        'includeTest': include_test,
        'includeFamily': include_family,
        'includeSpellcheck': include_spellcheck,
        'locale': locale,
        'preferredCountry': preferred_country
    }

    # Helper function to add parameters if they're provided
    def add_param(param_name, value, api_param_name=None):
        if value is not None:
            params[api_param_name or param_name] = value

    # Add identification parameters
    add_param('id', id)
    add_param('keyword', keyword)
    add_param('attractionId', attraction_id)
    add_param('venueId', venue_id)

    # Add geographic parameters
    add_param('countryCode', country_code)
    add_param('city', city)
    add_param('stateCode', state_code)
    add_param('postalCode', postal_code)
    add_param('latlong', latlong)
    add_param('geoPoint', geo_point)
    add_param('radius', radius)
    add_param('unit', unit)
    add_param('dmaId', dma_id)

    # Add date/time parameters
    add_param('startDateTime', start_date_time)
    add_param('endDateTime', end_date_time)
    add_param('localStartDateTime', local_start_date_time)
    add_param('localStartEndDateTime', local_start_end_date_time)
    add_param('startEndDateTime', start_end_date_time)

    # Add classification parameters
    add_param('classificationName', classification_name)
    add_param('classificationId', classification_id)
    add_param('segmentId', segment_id)
    add_param('segmentName', segment_name)
    add_param('genreId', genre_id)
    add_param('subGenreId', sub_genre_id)
    add_param('typeId', type_id)
    add_param('subTypeId', sub_type_id)

    # Add sale parameters
    add_param('onsaleStartDateTime', onsale_start_date_time)
    add_param('onsaleEndDateTime', onsale_end_date_time)
    add_param('onsaleOnStartDate', onsale_on_start_date)
    add_param('onsaleOnAfterStartDate', onsale_on_after_start_date)
    add_param('preSaleDateTime', pre_sale_date_time)
    add_param('publicVisibilityStartDateTime', public_visibility_start_date_time)

    # Add other filtering parameters
    add_param('marketId', market_id)
    add_param('promoterId', promoter_id)
    add_param('collectionId', collection_id)
    add_param('source', source)
    add_param('domain', domain)

    # Make the API request
    response = requests.get(base_url, params=params)

    # Check if the request was successful
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code}")
        print(response.text)
        return None

# Process the events data into structured format
def process_events(events_data):
    if not events_data or '_embedded' not in events_data:
        print("No events found or invalid response")
        return None

    event_list = []

    for event in events_data['_embedded']['events']:
        # Extract basic event info
        event_info = {
            'id': event['id'],
            'name': event['name'],
            'url': event['url'],
            'status': event['dates']['status']['code'] if 'status' in event['dates'] else 'unknown',
            'start_date': event['dates']['start'].get('localDate', 'unknown'),
            'start_time': event['dates']['start'].get('localTime', 'unknown'),
        }

        # Add end date and time if available
        if 'end' in event['dates']:
            event_info['end_date'] = event['dates']['end'].get('localDate', 'unknown')
            event_info['end_time'] = event['dates']['end'].get('localTime', 'unknown')

        # Calculate duration if possible
        if ('end' in event['dates'] and
            event_info['start_date'] != 'unknown' and event_info['start_time'] != 'unknown' and
            event_info['end_date'] != 'unknown' and event_info['end_time'] != 'unknown'):
            try:
                start_dt = datetime.strptime(f"{event_info['start_date']} {event_info['start_time']}", "%Y-%m-%d %H:%M:%S")
                end_dt = datetime.strptime(f"{event_info['end_date']} {event_info['end_time']}", "%Y-%m-%d %H:%M:%S")
                duration = end_dt - start_dt
                event_info['duration_minutes'] = duration.total_seconds() / 60
            except:
                # Handle cases where time formats might be different
                event_info['duration_minutes'] = 'unknown'

        # Add venue info if available
        if '_embedded' in event and 'venues' in event['_embedded'] and len(event['_embedded']['venues']) > 0:
            venue = event['_embedded']['venues'][0]
            event_info['venue_id'] = venue.get('id', 'unknown')
            event_info['venue_name'] = venue.get('name', 'unknown')
            event_info['venue_city'] = venue.get('city', {}).get('name', 'unknown')
            event_info['venue_state'] = venue.get('state', {}).get('name', 'unknown')
            event_info['venue_country'] = venue.get('country', {}).get('name', 'unknown')
            event_info['venue_postal_code'] = venue.get('postalCode', 'unknown')
            event_info['venue_url'] = venue.get('url', 'unknown')

            # Add venue coordinates if available
            if 'location' in venue:
                event_info['venue_latitude'] = venue['location'].get('latitude', 'unknown')
                event_info['venue_longitude'] = venue['location'].get('longitude', 'unknown')

        # Add price range if available
        if 'priceRanges' in event and len(event['priceRanges']) > 0:
            price_range = event['priceRanges'][0]
            event_info['min_price'] = price_range.get('min', 'unknown')
            event_info['max_price'] = price_range.get('max', 'unknown')
            event_info['currency'] = price_range.get('currency', 'unknown')

        # Add genre/classification if available
        if 'classifications' in event and len(event['classifications']) > 0:
            classification = event['classifications'][0]
            if 'segment' in classification and classification['segment']:
                event_info['segment'] = classification['segment'].get('name', 'unknown')
                event_info['segment_id'] = classification['segment'].get('id', 'unknown')
            if 'genre' in classification and classification['genre']:
                event_info['genre'] = classification['genre'].get('name', 'unknown')
                event_info['genre_id'] = classification['genre'].get('id', 'unknown')
            if 'subGenre' in classification and classification['subGenre']:
                event_info['subgenre'] = classification['subGenre'].get('name', 'unknown')
                event_info['subgenre_id'] = classification['subGenre'].get('id', 'unknown')
            if 'type' in classification and classification['type']:
                event_info['type'] = classification['type'].get('name', 'unknown')
                event_info['type_id'] = classification['type'].get('id', 'unknown')
            if 'subType' in classification and classification['subType']:
                event_info['subtype'] = classification['subType'].get('name', 'unknown')
                event_info['subtype_id'] = classification['subType'].get('id', 'unknown')

        # Add image URLs if available
        if 'images' in event and len(event['images']) > 0:
            # Find standard image
            standard_images = [img for img in event['images'] if img.get('ratio') == '16_9' and img.get('width') > 500]
            if standard_images:
                event_info['image_url'] = standard_images[0]['url']
            else:
                event_info['image_url'] = event['images'][0]['url']  # Fallback to first image

        # Add attractions/performers if available
        if '_embedded' in event and 'attractions' in event['_embedded'] and len(event['_embedded']['attractions']) > 0:
            attractions = []
            attraction_ids = []
            for attraction in event['_embedded']['attractions']:
                attractions.append(attraction.get('name', 'unknown'))
                attraction_ids.append(attraction.get('id', 'unknown'))
            event_info['attractions'] = ', '.join(attractions)
            event_info['attraction_ids'] = ', '.join(attraction_ids)

        # Add ticket info if available
        if 'sales' in event and 'public' in event['sales']:
            sales_public = event['sales']['public']
            if 'startDateTime' in sales_public:
                event_info['on_sale_date'] = sales_public['startDateTime']
            if 'endDateTime' in sales_public:
                event_info['off_sale_date'] = sales_public['endDateTime']

        # Add pre-sale info if available
        if 'sales' in event and 'presales' in event['sales'] and len(event['sales']['presales']) > 0:
            presale = event['sales']['presales'][0]  # Get the first pre-sale info
            event_info['presale_name'] = presale.get('name', 'unknown')
            event_info['presale_start_date'] = presale.get('startDateTime', 'unknown')
            event_info['presale_end_date'] = presale.get('endDateTime', 'unknown')

        event_list.append(event_info)

    # Create DataFrame
    df = pd.DataFrame(event_list)
    return df

# Function to fetch events with pagination
def fetch_all_events(max_pages=5, events_per_page=20, **search_params):
    all_events = []
    total_elements = 0
    total_pages = 0

    for page in range(max_pages):
        # Update the page parameter
        search_params['page'] = page
        search_params['size'] = events_per_page

        # Fetch the current page
        events_data = fetch_events(**search_params)

        # Check if we have a valid response
        if events_data and '_embedded' in events_data and 'events' in events_data['_embedded']:
            # Get page info
            page_info = events_data.get('page', {})
            total_elements = page_info.get('totalElements', 0)
            total_pages = page_info.get('totalPages', 0)

            # Process the events and add them to our collection
            page_events = events_data['_embedded']['events']
            all_events.extend(page_events)

            print(f"Fetched page {page+1}/{total_pages} ({len(page_events)} events)")

            # Check if we've reached the last page
            if page >= total_pages - 1:
                break
        else:
            print(f"No events found on page {page+1} or error in response")
            break

    # Convert all events to raw format for processing
    raw_events_data = {
        '_embedded': {
            'events': all_events
        }
    }

    print(f"Total events fetched: {len(all_events)} out of {total_elements}")
    return raw_events_data

# Function to display results as table and JSON
def display_structured_events(events_data):
    # Process the events
    events_df = process_events(events_data)

    if events_df is not None and not events_df.empty:
        # Display info
        print(f"Found {len(events_df)} events")

        # Display as styled table
        print("\n=== EVENTS TABLE ===")
        display(events_df)

        # Convert to JSON format and display
        print("\n=== EVENTS JSON ===")
        events_json = events_df.to_dict(orient='records')
        print(json.dumps(events_json, indent=2))

        return {
            'dataframe': events_df,
            'json': events_json
        }
    else:
        print("No events to display")
        return None

# Save results to CSV, JSON, or Excel
def save_results(results, format='all', prefix='events'):
    if results is None or 'dataframe' not in results:
        print("No results to save")
        return

    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    df = results['dataframe']

    if format.lower() in ['csv', 'all']:
        filename = f"{prefix}_{timestamp}.csv"
        df.to_csv(filename, index=False)
        print(f"Saved CSV to {filename}")

    if format.lower() in ['json', 'all']:
        filename = f"{prefix}_{timestamp}.json"
        with open(filename, 'w') as f:
            json.dump(results['json'], f, indent=2)
        print(f"Saved JSON to {filename}")

    if format.lower() in ['excel', 'all']:
        filename = f"{prefix}_{timestamp}.xlsx"
        df.to_excel(filename, index=False)
        print(f"Saved Excel to {filename}")

# Example usage
def main():
    # Define your search parameters with more options
    params = {
        'country_code': "US",
        'classification_name': "sports",  # Music, Sports, Arts, Film, etc.
        'city': "Los Angeles",
        'state_code': "CA",
        'start_date_time': "2025-05-15T00:00:00Z",  # Events after this date
        'end_date_time': "2025-08-15T23:59:59Z",    # Events before this date
        'sort': "date,asc",                        # Sort by date ascending
        'include_family': "yes"                    # Include family-friendly events
    }

    # Fetch events with pagination (up to 3 pages, 20 events per page)
    print(f"Fetching {params['classification_name']} events in {params['city']}, {params['state_code']}...")
    print(f"Time range: {params['start_date_time']} to {params['end_date_time']}")

    events_data = fetch_all_events(max_pages=3, events_per_page=20, **params)

    # Display structured results
    results = display_structured_events(events_data)

    # Save results to files
    save_results(results, format='all', prefix='la_sports_events')

    return results

# Run the main function
if __name__ == "__main__":
    results = main()

# fetch all available event details

In [ ]:
from google.colab import userdata

import requests
import pandas as pd
import json
from datetime import datetime
from IPython.display import display, HTML

api_key = userdata.get('TICKETMASTER_API_KEY')
base_url = "https://app.ticketmaster.com/discovery/v2/events.json"

def fetch_events(
    # Basic parameters
    country_code=None,
    size=20,
    page=0,
    sort="date,asc",

    # Identification parameters
    id=None,
    keyword=None,
    attraction_id=None,
    venue_id=None,

    # Geographic parameters
    city=None,
    state_code=None,
    postal_code=None,
    latlong=None,
    geo_point=None,
    radius=None,
    unit="miles",
    dma_id=None,

    # Date and time parameters
    start_date_time=None,
    end_date_time=None,
    local_start_date_time=None,
    local_start_end_date_time=None,
    start_end_date_time=None,

    # Classification parameters
    classification_name=None,
    classification_id=None,
    segment_id=None,
    segment_name=None,
    genre_id=None,
    sub_genre_id=None,
    type_id=None,
    sub_type_id=None,

    # Sale parameters
    onsale_start_date_time=None,
    onsale_end_date_time=None,
    onsale_on_start_date=None,
    onsale_on_after_start_date=None,
    pre_sale_date_time=None,
    public_visibility_start_date_time=None,

    # Include/exclude parameters
    include_tba="yes",
    include_tbd="yes",
    include_test="no",
    include_family="yes",
    include_spellcheck="no",

    # Other filtering parameters
    market_id=None,
    promoter_id=None,
    collection_id=None,
    source=None,
    locale="en",
    domain=None,
    preferred_country="us"
):
    # Define parameters dict with base parameters
    params = {
        'apikey': api_key,
        'size': size,
        'page': page,
        'sort': sort,
        'includeTBA': include_tba,
        'includeTBD': include_tbd,
        'includeTest': include_test,
        'includeFamily': include_family,
        'includeSpellcheck': include_spellcheck,
        'locale': locale,
        'preferredCountry': preferred_country
    }

    # Helper function to add parameters if they're provided
    def add_param(param_name, value, api_param_name=None):
        if value is not None:
            params[api_param_name or param_name] = value

    # Add identification parameters
    add_param('id', id)
    add_param('keyword', keyword)
    add_param('attractionId', attraction_id)
    add_param('venueId', venue_id)

    # Add geographic parameters
    add_param('countryCode', country_code)
    add_param('city', city)
    add_param('stateCode', state_code)
    add_param('postalCode', postal_code)
    add_param('latlong', latlong)
    add_param('geoPoint', geo_point)
    add_param('radius', radius)
    add_param('unit', unit)
    add_param('dmaId', dma_id)

    # Add date/time parameters
    add_param('startDateTime', start_date_time)
    add_param('endDateTime', end_date_time)
    add_param('localStartDateTime', local_start_date_time)
    add_param('localStartEndDateTime', local_start_end_date_time)
    add_param('startEndDateTime', start_end_date_time)

    # Add classification parameters
    add_param('classificationName', classification_name)
    add_param('classificationId', classification_id)
    add_param('segmentId', segment_id)
    add_param('segmentName', segment_name)
    add_param('genreId', genre_id)
    add_param('subGenreId', sub_genre_id)
    add_param('typeId', type_id)
    add_param('subTypeId', sub_type_id)

    # Add sale parameters
    add_param('onsaleStartDateTime', onsale_start_date_time)
    add_param('onsaleEndDateTime', onsale_end_date_time)
    add_param('onsaleOnStartDate', onsale_on_start_date)
    add_param('onsaleOnAfterStartDate', onsale_on_after_start_date)
    add_param('preSaleDateTime', pre_sale_date_time)
    add_param('publicVisibilityStartDateTime', public_visibility_start_date_time)

    # Add other filtering parameters
    add_param('marketId', market_id)
    add_param('promoterId', promoter_id)
    add_param('collectionId', collection_id)
    add_param('source', source)
    add_param('domain', domain)

    # Make the API request
    response = requests.get(base_url, params=params)

    # Check if the request was successful
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code}")
        print(response.text)
        return None

# Process the events data into structured format
def process_events(events_data):
    if not events_data or '_embedded' not in events_data:
        print("No events found or invalid response")
        return None

    event_list = []

    for event in events_data['_embedded']['events']:
        # Extract basic event info
        event_info = {
            'id': event['id'],
            'name': event['name'],
            #'url': event['url'],
            'status': event['dates']['status']['code'] if 'status' in event['dates'] else 'unknown',
            'start_date': event['dates']['start'].get('localDate', 'unknown'),
            'start_time': event['dates']['start'].get('localTime', 'unknown'),
        }

        # Add end date and time if available
        if 'end' in event['dates']:
            event_info['end_date'] = event['dates']['end'].get('localDate', 'unknown')
            event_info['end_time'] = event['dates']['end'].get('localTime', 'unknown')

        # Calculate duration if possible
        if ('end' in event['dates'] and
            event_info['start_date'] != 'unknown' and event_info['start_time'] != 'unknown' and
            event_info['end_date'] != 'unknown' and event_info['end_time'] != 'unknown'):
            try:
                start_dt = datetime.strptime(f"{event_info['start_date']} {event_info['start_time']}", "%Y-%m-%d %H:%M:%S")
                end_dt = datetime.strptime(f"{event_info['end_date']} {event_info['end_time']}", "%Y-%m-%d %H:%M:%S")
                duration = end_dt - start_dt
                event_info['duration_minutes'] = duration.total_seconds() / 60
            except:
                # Handle cases where time formats might be different
                event_info['duration_minutes'] = 'unknown'

        # Add venue info if available
        if '_embedded' in event and 'venues' in event['_embedded'] and len(event['_embedded']['venues']) > 0:
            venue = event['_embedded']['venues'][0]
            event_info['venue_id'] = venue.get('id', 'unknown')
            event_info['venue_name'] = venue.get('name', 'unknown')
            event_info['venue_city'] = venue.get('city', {}).get('name', 'unknown')
            event_info['venue_state'] = venue.get('state', {}).get('name', 'unknown')
            event_info['venue_country'] = venue.get('country', {}).get('name', 'unknown')
            event_info['venue_postal_code'] = venue.get('postalCode', 'unknown')
            event_info['venue_url'] = venue.get('url', 'unknown')

            # Add venue coordinates if available
            if 'location' in venue:
                event_info['venue_latitude'] = venue['location'].get('latitude', 'unknown')
                event_info['venue_longitude'] = venue['location'].get('longitude', 'unknown')

        # Add price range if available
        if 'priceRanges' in event and len(event['priceRanges']) > 0:
            price_range = event['priceRanges'][0]
            event_info['min_price'] = price_range.get('min', 'unknown')
            event_info['max_price'] = price_range.get('max', 'unknown')
            event_info['currency'] = price_range.get('currency', 'unknown')

        # Add genre/classification if available
        if 'classifications' in event and len(event['classifications']) > 0:
            classification = event['classifications'][0]
            if 'segment' in classification and classification['segment']:
                event_info['segment'] = classification['segment'].get('name', 'unknown')
                event_info['segment_id'] = classification['segment'].get('id', 'unknown')
            if 'genre' in classification and classification['genre']:
                event_info['genre'] = classification['genre'].get('name', 'unknown')
                event_info['genre_id'] = classification['genre'].get('id', 'unknown')
            if 'subGenre' in classification and classification['subGenre']:
                event_info['subgenre'] = classification['subGenre'].get('name', 'unknown')
                event_info['subgenre_id'] = classification['subGenre'].get('id', 'unknown')
            if 'type' in classification and classification['type']:
                event_info['type'] = classification['type'].get('name', 'unknown')
                event_info['type_id'] = classification['type'].get('id', 'unknown')
            if 'subType' in classification and classification['subType']:
                event_info['subtype'] = classification['subType'].get('name', 'unknown')
                event_info['subtype_id'] = classification['subType'].get('id', 'unknown')

        # Add image URLs if available
        if 'images' in event and len(event['images']) > 0:
            # Find standard image
            standard_images = [img for img in event['images'] if img.get('ratio') == '16_9' and img.get('width') > 500]
            if standard_images:
                event_info['image_url'] = standard_images[0]['url']
            else:
                event_info['image_url'] = event['images'][0]['url']  # Fallback to first image

        # Add attractions/performers if available
        if '_embedded' in event and 'attractions' in event['_embedded'] and len(event['_embedded']['attractions']) > 0:
            attractions = []
            attraction_ids = []
            for attraction in event['_embedded']['attractions']:
                attractions.append(attraction.get('name', 'unknown'))
                attraction_ids.append(attraction.get('id', 'unknown'))
            event_info['attractions'] = ', '.join(attractions)
            event_info['attraction_ids'] = ', '.join(attraction_ids)

        # Add ticket info if available
        if 'sales' in event and 'public' in event['sales']:
            sales_public = event['sales']['public']
            if 'startDateTime' in sales_public:
                event_info['on_sale_date'] = sales_public['startDateTime']
            if 'endDateTime' in sales_public:
                event_info['off_sale_date'] = sales_public['endDateTime']

        # Add pre-sale info if available
        if 'sales' in event and 'presales' in event['sales'] and len(event['sales']['presales']) > 0:
            presale = event['sales']['presales'][0]  # Get the first pre-sale info
            event_info['presale_name'] = presale.get('name', 'unknown')
            event_info['presale_start_date'] = presale.get('startDateTime', 'unknown')
            event_info['presale_end_date'] = presale.get('endDateTime', 'unknown')

        event_list.append(event_info)

    # Create DataFrame
    df = pd.DataFrame(event_list)
    return df

# Function to fetch events with pagination
def fetch_all_events(max_pages=5, events_per_page=20, **search_params):
    all_events = []
    total_elements = 0
    total_pages = 0
    pagination_info = {}

    for page in range(max_pages):
        # Update the page parameter
        search_params['page'] = page
        search_params['size'] = events_per_page

        # Fetch the current page
        events_data = fetch_events(**search_params)

        # Check if we have a valid response
        if events_data and '_embedded' in events_data and 'events' in events_data['_embedded']:
            # Get page info
            page_info = events_data.get('page', {})
            current_page = page_info.get('number', page)
            events_per_page_actual = page_info.get('size', events_per_page)
            total_elements = page_info.get('totalElements', 0)
            total_pages = page_info.get('totalPages', 0)

            # Store pagination information (from the first page)
            if page == 0:
                pagination_info = {
                    'events_per_page': events_per_page_actual,
                    'total_events': total_elements,
                    'total_pages': total_pages
                }

                # Print pagination information
                print(f"\n=== PAGINATION INFORMATION ===")
                print(f"Events per page: {events_per_page_actual}")
                print(f"Total events available: {total_elements}")
                print(f"Total pages available: {total_pages}")
                print(f"Maximum pages to fetch: {min(max_pages, total_pages)}")
                print("===============================\n")

            # Process the events and add them to our collection
            page_events = events_data['_embedded']['events']
            all_events.extend(page_events)

            print(f"Fetched page {current_page+1}/{total_pages} ({len(page_events)} events)")

            # Check if we've reached the last page
            if page >= total_pages - 1:
                break
        else:
            print(f"No events found on page {page+1} or error in response")
            break

    # Convert all events to raw format for processing
    raw_events_data = {
        '_embedded': {
            'events': all_events
        },
        'pagination_info': pagination_info
    }

    print(f"Total events fetched: {len(all_events)} out of {total_elements}")
    return raw_events_data

# Function to display results as table and JSON
def display_structured_events(events_data):
    # Process the events
    events_df = process_events(events_data)

    if events_df is not None and not events_df.empty:
        # Display info
        print(f"Found {len(events_df)} events")

        # Display pagination info if available
        if 'pagination_info' in events_data:
            pagination = events_data['pagination_info']
            print("\n=== PAGINATION SUMMARY ===")
            print(f"Events per page: {pagination.get('events_per_page', 'Unknown')}")
            print(f"Total events available: {pagination.get('total_events', 'Unknown')}")
            print(f"Total pages available: {pagination.get('total_pages', 'Unknown')}")
            print("===========================")

        # Display as styled table
        #print("\n=== EVENTS TABLE ===")
        #display(events_df)

        # Convert to JSON format and display
        print("\n=== EVENTS JSON ===")
        events_json = events_df.to_dict(orient='records')
        print(json.dumps(events_json, indent=2))

        # Create results dictionary including pagination info if available
        results = {
            'dataframe': events_df,
            'json': events_json
        }

        if 'pagination_info' in events_data:
            results['pagination'] = events_data['pagination_info']

        return results
    else:
        print("No events to display")
        return None

# Save results to CSV, JSON, or Excel
def save_results(results, format='all', prefix='events'):
    if results is None or 'dataframe' not in results:
        print("No results to save")
        return

    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    df = results['dataframe']

    if format.lower() in ['csv', 'all']:
        filename = f"{prefix}_{timestamp}.csv"
        df.to_csv(filename, index=False)
        print(f"Saved CSV to {filename}")

    if format.lower() in ['json', 'all']:
        filename = f"{prefix}_{timestamp}.json"
        with open(filename, 'w') as f:
            json.dump(results['json'], f, indent=2)
        print(f"Saved JSON to {filename}")

    if format.lower() in ['excel', 'all']:
        filename = f"{prefix}_{timestamp}.xlsx"
        df.to_excel(filename, index=False)
        print(f"Saved Excel to {filename}")

# Example usage
def main():
    # Define your search parameters with more options
    params = {
        'country_code': "US",
        'classification_name': "music",  # Music, Sports, Arts, Film, etc.
        'city': "New York",
        'state_code': "NY",
        'start_date_time': "2025-05-15T00:00:00Z",  # Events after this date
        #'end_date_time': "2025-08-15T23:59:59Z",    # Events before this date
        'sort': "date,asc",                        # Sort by date ascending
        #'include_family': "yes"                    # Include family-friendly events
    }

    # Define pagination parameters
    max_pages = 10     # Maximum number of pages to fetch
    events_per_page = 200  # Number of events per page to request

    # You can control the number of events per page by changing the 'size' parameter
    # The API default is 20, but you can set it up to the API's maximum (usually 50-100)
    params['size'] = 200 # Uncomment to override events_per_page

    # Fetch events with pagination
    #print(f"Fetching {params['classification_name']} events in {params['city']}, {params['state_code']}...")
    #print(f"Time range: {params['start_date_time']} to {params['end_date_time']}")
    #print(f"Pagination settings: max {max_pages} pages, {events_per_page} events per page")

    events_data = fetch_all_events(max_pages=max_pages, events_per_page=events_per_page, **params)

    # Display structured results
    results = display_structured_events(events_data)

    # Save results to files
    save_results(results, format='all', prefix='la_sports_events')

    # Return all the information
    if results and 'pagination' in results:
        print("\n=== PAGINATION SUMMARY ===")
        print(f"Events per page: {results['pagination']['events_per_page']}")
        print(f"Total events available: {results['pagination']['total_events']}")
        print(f"Total pages available: {results['pagination']['total_pages']}")

    return results

# Run the main function
if __name__ == "__main__":
    results = main()